In [3]:
!pip install "giskard[llm]" python-dotenv langchain langchain-openai langchain-community langchain_pinecone pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.3/603.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 30.4

In [19]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

from giskard.rag import KnowledgeBase, generate_testset, evaluate

In [5]:
# load_dotenv()
# OPEN_API_KEY = os.getenv("OPEN_API_KEY")
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

from google.colab import userdata
OPEN_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY

MODEL = "gpt-4o" # chat model
# MODEL = "llama3" # completion model

In [6]:
parser = StrOutputParser()
model = ChatOpenAI(model=MODEL, api_key=OPEN_API_KEY)
embeddings = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY)

In [7]:
chain = model | parser
chain.invoke("Tell me a joke")

"Sure, here's a joke for you:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!"

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
loader = WebBaseLoader("https://www.ml.school/")

docs = loader.load_and_split(splitter)
len(docs)

125

In [9]:
pinecone = PineconeVectorStore.from_documents(
    docs, embedding=embeddings, index_name="rag-evaluate-index"
)

## Knowledge Base

In [10]:
df = pd.DataFrame(data=[d.page_content for d in docs],columns=["text"])
df.head()

,text
0,Building Machine Learning Systems That Don't S...
1,"course I've done. Worth every cent.""Jose Reyes..."
2,Machine Learning Systems That Don't SuckA live...
3,help you build production-ready machine learni...
4,"cohort: August 5 - 22, 2024Check the schedule ..."


In [11]:
kb = KnowledgeBase(df)
kb

In [12]:
testset = generate_testset(
    kb,
    num_questions=60,
    agent_description="Chatbot to answer my linkedin page questions"
)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

ERROR:giskard.rag:Encountered error in question generation: list index out of range. Skipping.
ERROR:giskard.rag:list index out of range
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/giskard/rag/question_generators/base.py", line 57, in generate_questions
    yield self.generate_single_question(knowledge_base, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/giskard/rag/question_generators/double_questions.py", line 127, in generate_single_question
    "question_2": linked_questions[1]["question"],
IndexError: list index out of range


In [13]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(10).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

Question 1: Are there any monthly or annual fees to join?
Reference answer: No, there are no monthly fees, no annual fees, and no hidden costs. You pay once to join.
Reference context:
Document 25: join. There are no monthly fees. No annual fees. No hidden costs. You pay once to join and benefit
******************

Question 2: What principles should be considered before starting any project?
Reference answer: Before starting any project, you should consider problem framing, inversion, and the haystack principle.
Reference context:
Document 83: before starting any project.Problem framing, inversion, and the haystack principle for building
******************

Question 3: What is the advantage of having lifetime access?
Reference answer: The lifetime access removes any pressure from having to complete the
Reference context:
Document 115: to benefit from the updates.The lifetime access removes any pressure from having to complete the
******************

Question 4: What is the topic of Ses

In [14]:
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [15]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What programming language does the course teach in?")

'The course teaches in Python.'

## Evaluate model on test set

In [17]:
def answer_qn(ques, history=None):
  return chain.invoke(ques)

In [20]:
report = evaluate(answer_qn, testset=testset, knowledge_base=kb)

Asking questions to the agent:   0%|          | 0/59 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

In [21]:
display(report)

In [22]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.600000
conversational,0.300000
distracting element,0.000000
double,0.555556
simple,0.800000
situational,0.400000


In [23]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
cbf0f80c-c4ca-4927-b09a-b2795276cf3e,What is the advantage of having lifetime access?,The lifetime access removes any pressure from ...,Document 115: to benefit from the updates.The ...,[],"{'question_type': 'simple', 'seed_document_id'...",The advantage of having lifetime access is tha...,False,The agent's answer focuses on the financial be...
6237853b-0390-4fac-9981-7282c96c113a,What are some methods for handling missing val...,Missing values in data can be handled and inte...,Document 91: using normalization and standardi...,[],"{'question_type': 'simple', 'seed_document_id'...",Some methods for handling missing values in da...,False,The agent incorrectly mentioned normalization ...
65ee83c1-2e13-4713-a20b-0eee2dba669e,Can you specify the number of years the progra...,The program has 30 years of industry experience.,Document 53: 30 years in the industry.Check th...,[],"{'question_type': 'complex', 'seed_document_id...",I don't know.,False,The agent failed to provide the correct inform...
e7fc3358-5678-43c0-b46a-951d66854892,Could you specify what is the user's preferred...,The user loves to work with neural networks an...,Document 119: love neural networks. I love to ...,[],"{'question_type': 'complex', 'seed_document_id...",I don't know.,False,The agent failed to provide the correct inform...
405cb5d7-ac8f-4f08-9a3a-aa51bacc3d2a,What is the professional role held by Juan Olano?,Juan Olano is a Machine Learning Engineer.,"Document 98: it with complete confidence.""Juan...",[],"{'question_type': 'complex', 'seed_document_id...",I don't know.,False,The agent failed to provide the correct inform...
f052b4d0-c6bc-4759-b2ed-a43d50349af2,What are the key areas or concepts that the cl...,The class will focus on the fundamental ideas ...,Document 36: class. We'll move quickly over th...,[],"{'question_type': 'complex', 'seed_document_id...",I don't know.,False,The agent failed to provide any information re...
9b38cc1b-866b-42df-89c8-db6fce8d46d5,Given your experience in building and scaling ...,You'll collaborate with thousands of people fr...,Document 22: where you'll collaborate with tho...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent failed to provide any information re...
bc8b796d-972c-44ab-86b0-53e865ba66db,Given that you've built products for companies...,"Yes, you can catch up asynchronously later usi...",Document 107: You can decide when to attend cl...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent failed to provide the correct inform...
5298440e-800d-404f-8660-71fd3a17c85a,What are the provided instructions intended fo...,The provided instructions are to help you buil...,Document 57: instructions to help you build a ...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent failed to provide the correct inform...
